In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time
import os

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": os.path.abspath("./downloads"),  # 다운로드 경로 지정
    "download.prompt_for_download": False,
    "safebrowsing.enabled": True
})

# ChromeDriver 경로
driver_path = r"C:\chromedriver-win64\chromedriver.exe"  # chromedriver 경로 확인
driver = webdriver.Chrome(service=Service(driver_path), options=chrome_options)

# 사이트 접속

# 다운로드 폴더 생성
if not os.path.exists("downloads"):
    os.makedirs("downloads")

wait = WebDriverWait(driver, 10)

# 페이지별 URL과 범위 설정
page_ranges = [
    {"url": "loan_household_new.php", "start": (2022, 8), "end": (2025, 8)},
    {"url": "loan_household_new_202202.php", "start": (2021, 2), "end": (2022, 7)},
    {"url": "loan_household.php", "start": (2019, 7), "end": (2021, 1)},
    {"url": "loan_household_5.php", "start": (2015, 9), "end": (2019, 6)},
    {"url": "loan_household_2.php", "start": (2010, 1), "end": (2015, 8)},  # 이전 데이터
]

# 연도/월 반복
for year in range(2015, 2026):
    for month in range(1, 13):
        # 현재 연월
        ym = (year, month)

        # 해당 월이 존재하는 페이지 찾기
        page_url = None
        for pr in page_ranges:
            if pr["start"] <= ym <= pr["end"]:
                page_url = pr["url"]
                break
        if not page_url:
            print(f"{year}-{month} 범위 데이터 없음, 스킵")
            continue

        # 페이지 접속
        driver.get(f"https://portal.kfb.or.kr/compare/{page_url}")
        time.sleep(2)

        # 이하 기존 코드 그대로
        # 연도/월 선택
        year_select = Select(driver.find_element(By.ID, "year"))
        year_select.select_by_value(str(year))
        time.sleep(0.3)

        month_select = Select(driver.find_element(By.ID, "month"))
        month_select.select_by_value(str(month).zfill(2))
        time.sleep(0.3)

        # 라디오 버튼 클릭
        try : 
            driver.find_element(By.ID, "select_new_balance_1").click()
            time.sleep(0.2)
        except : 
            pass
        try : 
            driver.find_element(By.ID, "opt_1_2").click()
            time.sleep(0.2)
        except : 
            pass
        try : 
            driver.find_element(By.ID, "all_show_1").click()
            time.sleep(0.2)
        except : 
            pass
        
        try : 
        # 전체 은행 선택
            bank_all_checkbox = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "BankAll"))
            )
        except : 
            pass
        driver.execute_script("arguments[0].click();", bank_all_checkbox)

        # 검색 버튼 클릭
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.LINK_TEXT, "검색"))
        )
        search_button.click()
        time.sleep(3)
        try : 
            # 엑셀 다운로드 클릭
            excel_button = driver.find_element(By.CSS_SELECTOR, "a.btn01[title='현재 검색 결과를 엑셀파일로 다운로드 합니다.']")
            driver.execute_script("arguments[0].click();", excel_button)
            time.sleep(5)

            latest_file = './downloads/가계대출금리[2015년 8월 이전](일시상환방식 주택담보대출 신용등급별 금리현황)_20250814.xlsx'
            new_path = latest_file.split('_')[0] + f'_{year}_{month}.xlsx'
            os.rename(latest_file, new_path)
        except : 
            print('ㅁ PROBLEM : ', ym)
            pass

print("모든 연도/월 다운로드 완료.")
driver.quit()
